### Imports and function definitions

In [24]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

In [25]:
# Run this if you want to build in 3.0 rather than 2.0
from config import Settings
Settings.uvcustommodulelocation='C:\\Users\\msubbarao\\SCISS\\Uniview Theater 3.0\\Custom Modules'

### USES Conf Template 

In [71]:
Template = """mesh
{  
    data  panaorama ./modules/$folderName/sphere.obj
    data  barrel ./modules/$folderName/GTA.uvstream
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec4f ColorMultiplier     $ColorMultiplier
            vec3f RotationAxis        $RotationAxis
            vec1f RotationAngle       $RotationAngle
            vec1f displayFraction   $displayFraction | public | desc "display fraction" | widget slider | range 0 1
            vec1f YearStart   $yearStart | public | desc "year to start spiral" | widget slider | range 1879 2018
            vec1f YearStart   $yearStart | public | desc "display fraction" | widget slider | range 1879 2018
            vec1f trailMin   $trailMin | public | desc "spiral trail alpha min" | widget slider | range 0 1
            vec1f trailLength   $trailLength | public | desc "spiral trail length" | widget slider | range 0 10
            vec1f spiralRadius  $spiralRadius | public | desc "spiral radius" | widget slider | range 0 50
            vec1f spiralOffset  $spiralOffset | public | desc "spiral height Offset" | widget slider | range -1 1
            vec1f spiralAngle  $spiralAngle | public | desc "spiral height scale" | widget slider | range 0 2
        }
    }
    pass
    {
        useDataObject panaorama
        shader
        {
            type defaultMeshShader
            {
                fragmentShader   ./modules/$folderName/co2Panorama.fs
                vertexShader ./modules/$folderName/equirectangularPanorama.vs
                
                texture panorama ./modules/$folderName/$panorama
                {
                }
                texture co2 ./modules/$folderName/co2.png
                {
                    magnify GL_NEAREST
                    minify GL_NEAREST                
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                }
                

                parameter1f Scale $Scale
                
                stateManagerVar __objectName__.ColorMultiplier      ColorMultiplier
                stateManagerVar __objectName__.RotationAxis         RotationAxis
                stateManagerVar __objectName__.RotationAngle        RotationAngle
                stateManagerVar __objectName__.displayFraction      displayFraction
                stateManagerVar __objectName__.Alpha                uv_alpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    #UV_CULL_MODE GL_FRONT
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
        pass
    {
        useDataObject barrel
        shader
        {
            type defaultMeshShader
            {
                fragmentShader   ./modules/$folderName/barrel.fs
                vertexShader ./modules/$folderName/barrel.vs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                }
                parameter1f Scale $Scale
                
                stateManagerVar __objectName__.yearStart            yearStart
                stateManagerVar __objectName__.trailMin      trailMin
                stateManagerVar __objectName__.trailLength      trailLength
                stateManagerVar __objectName__.spiralAngle      spiralAngle
                stateManagerVar __objectName__.spiralRadius      spiralRadius
                stateManagerVar __objectName__.spiralOffset      spiralOffset
                stateManagerVar __objectName__.RotationAxis         RotationAxis
                stateManagerVar __objectName__.RotationAngle        RotationAngle
                stateManagerVar __objectName__.Alpha                uv_alpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    #UV_CULL_MODE GL_FRONT
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                    UV_LINE_WIDTH 4
                }
            }

        }    
    }
    
}"""

### UVStream

In [31]:
UVSTREAM='''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<DataStream version="0.1">
    <PrimitiveType>line_strip</PrimitiveType>
    <DataBindings>
        <Variable>
            <Name>vertex</Name>         
            <Type>vec2f</Type>
        </Variable>
        <Variable>
            <Name>year</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>month</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>dT</Name>         
            <Type>float</Type>
        </Variable>             
    </DataBindings>

    <CSVData>
        <Path>GLB.Ts.UV.csv</Path>
        <Headers>
            <Variable>              
                <Cols>0,1</Cols>
            </Variable>
            <Variable>              
                <Cols>2</Cols>
            </Variable>                     
            <Variable>              
                <Cols>3</Cols>
            </Variable>                     
            <Variable>              
                <Cols>4</Cols>
            </Variable>                     
         </Headers>
    </CSVData>
</DataStream>  
'''

In [32]:
uvStreamFile=open('data/GTA.uvstream','w')
uvStreamFile.write(UVSTREAM)
uvStreamFile.close()

### KeelingChicago class

In [72]:
class KeelingChicago():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("sphere.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("equirectangularPanorama.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("co2Panorama.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("barrel.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("barrel.fs",dataFolder)
        self.panorama = "UNINITIALIZED"
        self.ColorMultiplier = "1 1 1 1"
        self.yearStart = 1975
        self.Scale = 1
        self.trailMin =0.2
        self.trailLength=1.0
        self.spiralAngle = 0.8
        self.spiralOffset = 0.0
        self.spiralRadius = 15.
        self.RotationAxis = "1 0 0"
        self.cr = 1000
        self.RotationAngle = 0
        self.displayFraction = 0.0
    def setPanorama(self, panorama):
        uvmod.Utility.ensurerelativepathexsists(panorama,dataFolder)
        self.panorama = panorama
    def setColorMultiplier(self,ColorMultiplier):
        self.ColorMultiplier = " ".join(str(c) for c in ColorMultiplier)
    def setScale(self,Scale):
        self.Scale = Scale
    def setRotationAxis(self,RotationAxis):
        self.RotationAxis = " ".join(str(c) for c in RotationAxis)
    def setRotationAngle(self,RotationAngle):
        self.RotationAngle = RotationAngle
    def setDisplayFraction(self,displayFraction):
        self.RotationAngle = displayFraction
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 panorama = self.panorama,
                                 ColorMultiplier = self.ColorMultiplier,
                                 trailMin = self.trailMin,
                                 trailLength = self.trailLength,
                                 spiralAngle = self.spiralAngle,
                                 spiralOffset= self.spiralOffset,
                                 spiralRadius = self.spiralRadius,
                                 Scale = self.Scale,
                                 displayFraction = self.displayFraction,
                                 yearStart=self.yearStart,
                                 RotationAxis = self.RotationAxis,
                                 RotationAngle = self.RotationAngle))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [73]:
model = KeelingChicago(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [74]:
model.object.setcameraradius(10)
model.object.setcoord("Earth")
model.object.setname("KeelingChicago")
model.object.setguiname("/KavliLecture/Schrag/KeelingChicago")
model.object.settargetradius(20)
model.object.showatstartup(False)
model.setPanorama("R0010133.JPG")
model.setColorMultiplier((1.,1.,1.,1.))
model.setScale(100)
model.cr = 10000
modinfo.setname("KeelingChicago")
modinfo.setauthor("Mark SubbaRao<sup>1</sup> and Daniel Schragg<sup>2</sup><br />(1)Adler Planetarium,br />(2)Harvard University")
modinfo.cleardependencies()
modinfo.setdesc("Atmospheric CO2 as red blow, temp anomoly as barrel visualization")
modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("KeelingChicago",[],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [129]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

Copy modified Shader files and reload

In [13]:
from config import Settings


In [55]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## CO2 data
This notebook process the data for the Uniview Keeling Curve module, the atmospheric concentration of carbon dioxide over time. The curve is named after atmospheric scientist Charles David Keeling. It utilizes data from NOAA's https://www.esrl.noaa.gov/gmd/dv/iadv/graph.php?code=MLO&program=ccgg&type=ts> Earth Systems Research Laboratory collected from their Mauna Loa Observatory. We'll use he in-situ monthly average measurements.

In [20]:
from astropy.io import ascii
from PIL import Image
import numpy as np

In [31]:
co2Table = ascii.read('rawdata/co2_mlo_surface-insitu_1_ccgg_MonthlyData.txt')

co2Table

col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17
str3,int32,int32,int32,int32,int32,int32,float64,float64,int32,float64,float64,float64,float64,float64,str2,str3
MLO,1973,1,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,2,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,3,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,4,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,5,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,6,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,7,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,8,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..
MLO,1973,9,1,0,0,0,-999.99,-99.99,0,19.536,-155.576,3437.0,3397.0,40.0,NA,*..


In [32]:
yearCol=co2Table['col2']+(co2Table['col3']-1.0)/12.

In [28]:
2018-1974

44

In [33]:
co2Texture = Image.new("RGB",(44,12))

In [34]:
for monthData in co2Table:
    iYear = monthData['col2']-1974
    iMonth = monthData['col3']-1
    valR=max(0,int(np.floor(monthData['col8'])-300))
    valG=int(256.0*(monthData['col8']-np.floor(monthData['col8'])))
    if (iYear>=0):
        co2Texture.putpixel((iYear,iMonth),(valR,valG,0))

In [35]:
co2Texture

In [36]:
co2Texture.save('data/co2.png')

## Global Temperature Anomoly

The dataset comes from the Goddard Institute for Space Studies: https://data.giss.nasa.gov/gistemp/

In [61]:
from astropy.io import ascii
from astropy.table import Table, Column

In [57]:
dtTable = ascii.read('rawdata/GLB.Ts.csv',header_start=1,data_start=2)

In [58]:
dtTable

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
int32,float64,str5,str4,str4,str5,str5,str4,str4,str4,str4,str4,str4,str4,str4,str4,str4,str4,str4
1880,-0.8,-.37,-.45,-.64,-.34,-.47,-.46,.10,-.47,-.65,-.49,-.51,-.46,***,***,-.47,-.28,-.54
1881,-0.76,-.59,-.34,-.24,.00,-1.09,-.50,-.21,-.31,-.45,-.53,-.11,-.43,-.46,-.62,-.19,-.60,-.43
1882,0.15,-.08,-.05,-.55,-.34,-.99,-.67,-.08,-.04,-.28,-.36,-.64,-.33,-.28,-.01,-.31,-.58,-.23
1883,-0.65,-.91,-.43,-.28,-.32,.47,.02,-.12,-.42,-.53,-.66,-.36,-.35,-.37,-.73,-.34,.12,-.54
1884,-0.57,-.31,-.37,-.91,-1.16,-.79,-.81,.19,-.33,-.74,-.78,-.96,-.63,-.58,-.41,-.81,-.47,-.62
1885,-0.89,-.76,-.88,-.83,-.45,-.90,-.59,-.12,-.12,.00,-.34,.05,-.49,-.57,-.87,-.72,-.54,-.15
1886,-0.72,-.79,-.65,-.58,-.53,-.81,-.36,-.69,-.38,-.82,-.45,-.71,-.63,-.56,-.49,-.59,-.62,-.55
1887,-0.84,-1.24,-.56,-.81,-.83,-.64,-.38,-.23,-.59,-.52,-.89,-.63,-.68,-.69,-.93,-.73,-.42,-.67
1888,-0.7,-.79,-.87,-.28,-.72,-.52,-.33,-.32,-.38,-.24,-.02,-.14,-.44,-.48,-.70,-.62,-.39,-.22


In [88]:
monthTable = Table(names=('theta', 'z', 'Year','Month','dT'))

In [90]:
for year in dtTable:
    for month in range(12):
        if (year[month+1]!='***'):
            monthTable.add_row([6.283*month/12.,year[month+1],year[0],month,year[month+1]])

In [94]:
monthTable.write('data/GLB.Ts.UV.csv',format='ascii.no_header',overwrite=True,delimiter=',')

### Create colormap texture

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('coolwarm')

plt.show()